In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Profile model training performance using Cloud Profiler in custom training with prebuilt container

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training_with_prebuilt_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftensorboard%2Ftensorboard_profiler_custom_training_with_prebuilt_container.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training_with_prebuilt_container.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training_with_prebuilt_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>
  

## Overview

The Profiler is a powerful tool that can help you to diagnose and debug performance bottlenecks, and make your model train faster. This tutorial demonstrates how to enable Profiler in Vertex AI for custom training with a prebuilt container.

Learn more about [Profiler](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler).

### Objective

In this tutorial, you learn how to enable Profiler in Vertex AI for custom training jobs with a prebuilt container.

This tutorial uses the following Google Cloud AI services:

- Vertex AI Training
- Vertex AI TensorBoard

The steps performed include:

- Prepare your custom training code and load your training code as a Python package to a prebuilt container
- Create and run a custom training job that enables Profiler
- View the Profiler dashboard to debug your model training performance


### Dataset

The dataset used for this tutorial is the [mnist dataset](https://www.tensorflow.org/datasets/catalog/mnist) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview).


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

**Setup service account and permissions**

A service account is used to create custom training jobs. If you do not want to use your project's Compute Engine service account, set SERVICE_ACCOUNT to another service account ID. You can create a service account by following the [instructions](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating).

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/storage.admin" \
        --quiet

# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/aiplatform.user" \
        --quiet

! gcloud projects get-iam-policy $PROJECT_ID \
        --filter=bindings.members:serviceAccount:$SERVICE_ACCOUNT

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Create a TensorBoard instance

A Vertex AI TensorBoard instance, which is a regionalized resource storing your Vertex AI TensorBoard experiments, must be created before the experiments can be visualized. You can create multiple instances in a project. You can use command  `gcloud ai tensorboards list` to get a list of your existing TensorBoard instances.

#### Set your TensorBoard instance display name


In [ ]:
TENSORBOARD_NAME = "your-tensorboard-unique"  # @param {type:"string"}

#### Create a TensorBoard instance

If you don't have a TensorBoard instance, create one by running the following cell:

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=LOCATION
)

TENSORBOARD_INSTANCE_NAME = tensorboard.resource_name
print("TensorBoard instance name:", TENSORBOARD_INSTANCE_NAME)

## Train a model

To train a model using your custom training code, choose one of the following options:

- **Prebuilt container**: Load your custom training code as a Python package to a prebuilt container image from Google Cloud.

- **Custom container**: Create your own container image that contains your custom training code.

In this tutorial, you will train a custom model using a prebuilt container.

### Examine the training package

#### Package layout

Before you start the training, let's take a look at how a Python package is assembled for a custom training job. When extracted, the package contains the following:

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the docker image.

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "app"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = f"{BUCKET_URI}/trainer-0.1.tar.gz"

# Make folder for Python training script
! rm -rf {PYTHON_PACKAGE_APPLICATION_DIR}
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}

# Add package information
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/README.md

# Make the training subfolder
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}/trainer
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/trainer/__init__.py

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess

REQUIRED_PACKAGES = [
    'google-cloud-aiplatform[cloud_profiler]>=1.20.0',
    'tensorflow==2.9.3',
    'protobuf>=3.9.2,<3.20'
]

setup(
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    name='trainer',
    version='0.1',
    url="wwww.google.com",
    description='Vertex AI | Training | Python Package'
)

#### Prepare the training script

The file `trainer/task.py` is the Python script for executing the custom training job.

Your training code must be configured to write TensorBoard logs to a Cloud Storage bucket, the location of which Vertex AI Training automatically makes available through a predefined environment variable, `AIP_TENSORBOARD_LOG_DIR`. This can usually be done by providing `os.environ['AIP_TENSORBOARD_LOG_DIR']` as the log directory to the open source TensorBoard log writing APIs. For example, in TensorFlow 2.x, you can use following code to create a `tensorboard_callback`:

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
      histogram_freq=1)
`AIP_TENSORBOARD_LOG_DIR` is in the `BASE_OUTPUT_DIR` that you provide when creating the custom training job.

To enable Profiler for your training job, add the following to your training script:

Add the cloud_profiler import at your top level imports:

    from google.cloud.aiplatform.training_utils import cloud_profiler


Initialize the cloud_profiler plugin by adding:


    cloud_profiler.init()

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/trainer/task.py

import tensorflow as tf
import argparse
import os
import sys, traceback
from google.cloud.aiplatform.training_utils import cloud_profiler

"""Train an mnist model and use cloud_profiler for profiling."""

def _create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def main(args):
    print('Initialize the profiler ...')
    cloud_profiler.init()
    print('The profiler initiated.')

    print('Loading and preprocessing data ...')
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print('Creating and training model ...')

    model = _create_model()
    model.compile(
      optimizer="adam",
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=["accuracy"],
    )

    log_dir = "logs"
    if 'AIP_TENSORBOARD_LOG_DIR' in os.environ:
      log_dir = os.environ['AIP_TENSORBOARD_LOG_DIR']

    print('Setting up the TensorBoard callback ...')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1)

    print('Training model ...')
    model.fit(
        x_train,
        y_train,
        epochs=args.epochs,
        verbose=0,
        callbacks=[tensorboard_callback],
    )
    print('Training completed.')

    print('Saving model ...')

    model_dir = "model"
    if 'AIP_MODEL_DIR' in os.environ:
      model_dir = os.environ['AIP_MODEL_DIR']
    tf.saved_model.save(model, model_dir)

    print('Model saved at ' + model_dir)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of epochs to run model."
    )

    args = parser.parse_args()
    main(args)

#### Create a source distribution

You create a source distribution with your training application and upload the source distribution to your Cloud Storage bucket.

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

!gsutil cp {source_package_file_name} {python_package_gcs_uri}

!gsutil ls -l {python_package_gcs_uri}

### Create and run the custom training job

Configure a [custom job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) with the [pre-built container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) image for training code packaged as Python source distribution.

In [ ]:
JOB_NAME = "tensorboard-job-unique"
MACHINE_TYPE = "n1-standard-4"
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest"
base_output_dir = f"{BUCKET_URI}/{JOB_NAME}"
python_module_name = "trainer.task"

EPOCHS = 20
training_args = [
    "--epochs=" + str(EPOCHS),
]

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME,
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=TRAIN_IMAGE,
)

#### Run the custom training job

Next, run the custom job to start the training job by invoking the method `run()`.

**NOTE:** When using Vertex AI SDK for Python for submitting a training job, it creates a [training pipeline](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) which launches the custom job on Vertex AI Training service.

In [ ]:
job.run(
    replica_count=1,
    machine_type=MACHINE_TYPE,
    base_output_dir=base_output_dir,
    tensorboard=TENSORBOARD_INSTANCE_NAME,
    service_account=SERVICE_ACCOUNT,
    args=training_args,
)

## View the Profiler dashboard

When the custom job state switches to running, you can access the Profiler dashboard through the Custom jobs page or the Experiments page on the Google Cloud console.

The Google Cloud guide to [Profile model training performance using Cloud Profiler](https://cloud.google.com/vertex-ai/docs/training/tensorboard-profiler) provides detailed instructions for accessing the Profiler dashboard and capturing a profiling session.


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
# delete training job
job.delete()

# delete tensorboard instance
tensorboard.delete()

# delete custom job
custom_job = f"{JOB_NAME}-custom-job"
custom_job_to_delete = aiplatform.CustomJob.list(filter=f"display_name={custom_job}")[0]
custom_job_to_delete.delete()

# delete locally generated files
! rm -rf {PYTHON_PACKAGE_APPLICATION_DIR}

# delete the bucket
delete_bucket = False  # set True for deletion
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI